In [1]:
# import importlib

import numpy as np
import datetime

import plugins.custom_config as custom_config
import plugins.bot_tool as bot_tool
# from bot_tool import *
import plugins.dice_tool as dice_tool
import plugins.help_info as help_info
# from dice_tool import *
import plugins.utils as utils

# importlib.reload(bot_tool)
# importlib.reload(dice_tool)
# importlib.reload(utils)

ujson module not found, using json


In [2]:
assert utils.ChineseToEnglishSymbol('。') == '.'
assert utils.ChineseToEnglishSymbol('.') == '.'
assert utils.int2str(4) == '+4'
assert utils.int2str(0) == '+0'
assert utils.int2str(-1) == '-1'

assert bot_tool.ParseInput('.rd20').equal(utils.Command(utils.CommandType.Roll, ['d20', '']))
assert bot_tool.ParseInput(' . r D20 reason').equal(utils.Command(utils.CommandType.Roll, ['d20', 'reason']))
assert bot_tool.ParseInput('.r').equal(utils.Command(utils.CommandType.Roll, ['', '']))
assert bot_tool.ParseInput('abc') is None
assert bot_tool.ParseInput('') is None
assert bot_tool.ParseInput('测试') is None

In [3]:
assert dice_tool.RollDiceCommond('12') [2][0] == 12 and dice_tool.RollDiceCommond('12') [1] == '12=12'
assert dice_tool.RollDiceCommond('4d6+2')[0] == 0 and dice_tool.RollDiceCommond('4d6+2') [1][:6] == '4D6+2='
assert dice_tool.RollDiceCommond('d20') [0] == 0 and dice_tool.RollDiceCommond('d20') [1][:3] == 'D20'
assert dice_tool.RollDiceCommond('d') [0] == 0 and dice_tool.RollDiceCommond('d') [1][:3] == 'D20'
assert dice_tool.RollDiceCommond('1D4') [0] == 0
assert dice_tool.RollDiceCommond('30D4') [0] == 0
assert dice_tool.RollDiceCommond('D20优势+2') [0] == 0 and dice_tool.RollDiceCommond('D20优势+2') [1][:12] == 'D20优势+2=Max('
assert dice_tool.RollDiceCommond('D20劣势-4') [0] == 0 and dice_tool.RollDiceCommond('D20劣势-4') [1][:12] == 'D20劣势-4=Min('
assert dice_tool.RollDiceCommond('4d6k3') [0] == 0 and dice_tool.RollDiceCommond('4d6k3') [1][:11] == '4D6K3=Max3('
assert dice_tool.RollDiceCommond('6#4d6k3') [0] == 0 and dice_tool.RollDiceCommond('6#4d6k3') [1][:14] == '6次4D6K3={\nMax3'
assert dice_tool.RollDiceCommond('2d20k1+5+1d4') [0] == 0 and dice_tool.RollDiceCommond('2d20k1+5+1d4') [1][:17] == '2D20K1+5+1D4=Max('

In [4]:
assert dice_tool.SplitDiceCommand('d20优势+5攻击地精') == ('d20优势+5', '攻击地精')
assert dice_tool.SplitDiceCommand('d20优势+5     攻击地精') == ('d20优势+5', '攻击地精')
assert dice_tool.SplitDiceCommand('d20 优 势+5     攻击地精') == ('d20优势+5', '攻击地精')
assert dice_tool.SplitDiceCommand('d20 优 势+5') == ('d20优势+5', '')

In [5]:
b = bot_tool.Bot()
assert b.ProcessInput('.nn', '12345678', 'ABCD') == '要用本来的名字称呼你吗? 了解!'
assert b.ProcessInput('.rd20', '12345678', 'TEST_USER')[:16] == 'TEST_USER掷出了D20='
assert b.ProcessInput('.rd', '12345678', 'TEST_USER')[:16] == 'TEST_USER掷出了D20='
assert b.ProcessInput('.rd     ', '12345678', 'TEST_USER')[:16] == 'TEST_USER掷出了D20='
assert b.ProcessInput('.rd    未知原因', '12345678', 'TEST_USER')[:23] == '由于未知原因,TEST_USER掷出了D20='
assert b.ProcessInput('.rd20攻击', '12345678', 'TEST_USER', 'groupId_test')[:21] == '由于攻击,TEST_USER掷出了D20='
assert b.ProcessInput('.rd优势', '12345678', 'TEST_USER')[:22] == 'TEST_USER掷出了D20优势=Max('
assert b.ProcessInput('.rd+5', '12345678', 'TEST_USER')[:18] == 'TEST_USER掷出了D20+5='

In [6]:
assert b.ProcessInput('.nn', '12345678', 'ABCD') == '要用本来的名字称呼你吗? 了解!'
assert b.ProcessInput('.nn 西瓜', '12345678', 'ABCD') == '要称呼ABCD为西瓜吗? 没问题!'
assert b.ProcessInput('.rd20', '12345678', 'TEST_USER')[:9] == '西瓜掷出了D20='
assert b.ProcessInput('.nn 冬瓜', '12345678', 'ABCD') == '要称呼ABCD为冬瓜吗? 没问题!'
assert b.ProcessInput('.rd20', '12345678', 'TEST_USER')[:9] == '冬瓜掷出了D20='
assert b.ProcessInput('.jrrp', '12345678', 'TEST_USER')[:10] == '冬瓜今天走运的概率是'
assert b.ProcessInput('.nn', '12345678', 'ABCD') == '要用本来的名字称呼你吗? 了解!'
assert b.ProcessInput('.rd20', '12345678', 'TEST_USER')[:16] == 'TEST_USER掷出了D20='
assert b.ProcessInput('.jrrp', '12345678', 'TEST_USER')[:17] == 'TEST_USER今天走运的概率是'

In [7]:
b.ProcessInput('.sethp 40/50', '12345678', 'TEST_USER', 'groupId_test')

'成功将TEST_USER的生命值设置为40=40, 最大生命值是50'

In [8]:
assert b.ProcessInput('.nn 格鲁特', '12345678', 'TEST_USER') == '要称呼TEST_USER为格鲁特吗? 没问题!'
assert b.ProcessInput('.sethp 40', '12345678', 'TEST_USER', 'groupId_test') == '成功将格鲁特的生命值设置为40=40'
assert b.ProcessInput('.sethp 40/50', '12345678', 'TEST_USER', 'groupId_test') == '成功将格鲁特的生命值设置为40=40, 最大生命值是50'
assert b.ProcessInput('.init clr', '12345678', 'TEST_USER', 'groupId_test')
assert b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test') == '还没有做好先攻列表哦'
assert b.ProcessInput('.ri+2', '12345678', 'TEST_USER', 'groupId_test')[:12] == '格鲁特先攻:D20+2='
assert b.ProcessInput('.ri-1', '12345678', 'TEST_USER', 'groupId_test')[:12] == '格鲁特先攻:D20-1='
assert b.ProcessInput('.ri', '12345678', 'TEST_USER', 'groupId_test')[:10] == '格鲁特先攻:D20='
assert b.ProcessInput('.ri-2地精', '12345678', 'TEST_USER', 'groupId_test')[:11] == '地精先攻:D20-2='
assert b.ProcessInput('.ri劣势+1d4 地精', '12345678', 'TEST_USER', 'groupId_test')[:19] =='地精先攻:D20劣势+1D4=Min('
assert b.ProcessInput('.ri20 巢穴', '12345678', 'TEST_USER', 'groupId_test') == '巢穴先攻:20=20'
assert b.ProcessInput('.ri-5', '87654321', '米达尔', 'groupId_test')[:12] == '米达尔先攻:D20-5='
print(b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test'))
assert b.ProcessInput('.init del 地精', '12345678', 'TEST_USER', 'groupId_test') == '已经将地精从先攻列表中删除'
assert b.ProcessInput('.init del米达尔', '12345678', 'TEST_USER', 'groupId_test') == '已经将米达尔从先攻列表中删除'
print(b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test'))
assert b.ProcessInput('.init clr', '12345678', 'TEST_USER', 'groupId_test')
assert b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test') == '还没有做好先攻列表哦'
assert b.ProcessInput('.nn', personId = '12345678', personName = 'TEST_USER') == '要用本来的名字称呼你吗? 了解!'
assert b.ProcessInput('.rd20', '12345678', 'ABCD')[:11] == 'ABCD掷出了D20='
assert b.ProcessInput('.jrrp', '12345678', 'TEST_USER')[:17] == 'TEST_USER今天走运的概率是'
assert b.ProcessInput('.sethp', '12345678', 'TEST_USER', 'groupId_test') == '已经忘记了TEST_USER的生命值...'

先攻列表:
1.巢穴 先攻20 HP:0
2.格鲁特 先攻15 HP:40/50
3.米达尔 先攻10 HP:0
4.地精 先攻4 HP:0
先攻列表:
1.巢穴 先攻20 HP:0
2.格鲁特 先攻15 HP:40/50


In [9]:
assert b.ProcessInput('.nn 格鲁特', '12345678', 'TEST_USER') == '要称呼TEST_USER为格鲁特吗? 没问题!'
print(b.ProcessInput('.ri', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.ri', '87654321', '米达尔', 'groupId_test'))
print(b.ProcessInput('.ri 地精', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.ri 卓尔精灵A', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethp 20/40', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethp 地精- 1d8+4', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethp 格+ 1d8+3', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethp10', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethp A-1d6+10', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethpA2d8', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethpA-20', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethp米-3d4+3', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.init', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethp', '12345678', 'TEST_USER', 'groupId_test'))
print(b.ProcessInput('.sethp', '87654321', '米达尔', 'groupId_test'))
assert b.ProcessInput('.nn', '12345678', 'TEST_USER') == '要用本来的名字称呼你吗? 了解!'

格鲁特先攻:D20=13
米达尔先攻:D20=6
地精先攻:D20=13
卓尔精灵a先攻:D20=1
先攻列表:
1.格鲁特 先攻13 HP:0
2.地精 先攻13 HP:0
3.米达尔 先攻6 HP:0
4.卓尔精灵a 先攻1 HP:0
成功将格鲁特的生命值设置为20=20, 最大生命值是40
地精的生命值减少了1D8+4=8+4=12
格鲁特的生命值增加了1D8+3=5+3=8
成功将格鲁特的生命值设置为10=10
卓尔精灵a的生命值减少了1D6+10=6+10=16
成功将卓尔精灵a的生命值设置为2D8=(3+3)=6
卓尔精灵a的生命值降至0, 卓尔精灵a昏迷/死亡了!
米达尔的生命值减少了3D4+3=(1+4+4)+3=12
先攻列表:
1.格鲁特 先攻13 HP:10/40
2.地精 先攻13 已损失HP:12
3.米达尔 先攻6 已损失HP:12
4.卓尔精灵a 先攻1 昏迷/死亡
已经忘记了格鲁特的生命值...
已经忘记了米达尔的生命值...


In [10]:
assert b.ProcessInput('.bot on', '12345678', 'TEST_USER', 'groupId_test') == None
assert b.ProcessInput('.bot on', '12345678', 'TEST_USER', 'groupId_test', True) == '来啦~ 让我看看哪个小可爱能得到今日份的礼物'
assert b.ProcessInput('.bot off', '12345678', 'TEST_USER', 'groupId_test', True) == '那我就不说话咯'
assert b.ProcessInput('.rd20沉默', '12345678', 'TEST_USER', 'groupId_test') == None
assert b.ProcessInput('.bot on', '12345678', 'TEST_USER', 'groupId_test', True) == '来啦~ 让我看看哪个小可爱能得到今日份的礼物'
assert b.ProcessInput('.rd20沉默', '12345678', 'TEST_USER', 'groupId_test') != None

In [11]:
assert b.ProcessInput('.dnd', '12345678', 'TEST_USER')
assert b.ProcessInput('.dnd 3', '12345678', 'TEST_USER', 'groupId_test')
assert b.ProcessInput('.dnd3 大力出奇迹', '12345678', 'TEST_USER', 'groupId_test')

In [12]:
assert b.ProcessInput('.help', '12345678', 'TEST_USER', 'groupId_test') == help_info.HELP_STR
assert b.ProcessInput('.help指令', '12345678', 'TEST_USER', 'groupId_test') == help_info.HELP_COMMAND_STR
assert b.ProcessInput('.help  指 令', '12345678', 'TEST_USER', 'groupId_test') == help_info.HELP_COMMAND_STR
assert b.ProcessInput('.help链接', '12345678', 'TEST_USER', 'groupId_test') == help_info.HELP_LINK_STR
assert b.ProcessInput('.help协议', '12345678', 'TEST_USER', 'groupId_test') == help_info.HELP_AGREEMENT_STR

In [13]:
assert b.ProcessInput('.查询火球术', '12345678', 'TEST_USER', 'groupId_test')[:11] == '火球术Fireball'
assert b.ProcessInput('.查询  火球术', '12345678', 'TEST_USER', 'groupId_test')[:11] == '火球术Fireball'
assert b.ProcessInput('.查询  火 球 术', '12345678', 'TEST_USER', 'groupId_test')[:11] == '火球术Fireball'
assert b.ProcessInput('.查询火球', '12345678', 'TEST_USER', 'groupId_test').find('找到多个匹配的条目:') != -1
assert b.ProcessInput('.查询   ', '12345678', 'TEST_USER', 'groupId_test')

In [19]:
# queryDict = {}

# title = ''
# content = ''
# with open('../5E_Magic_Items_by花作噫.txt', 'r') as f:
#     data = f.readline()
#     while data:
#         data = data.strip()
#         if not data:
#             if title:
#                 if len(title) > 10:
#                     print(title)
#                 elif len(content) < 20:
#                     print(title+"##"+content)
#                 queryDict[title] = content[:-1]
#             title = ''
#             content = ''
#         else:
#             if not title:
#                 title = data
#             else:
#                 content += data.strip() + '\n'
#         data = f.readline()
# print(queryDict.keys(), queryDict[list(queryDict.keys())[0]])
# utils.UpdateJson(queryDict, './temp.json')

dict_keys(['攀爬药水', '精金护甲', '炼金壶', '反侦护符', '次元袋', '魔术袋', '精灵之靴', '跳跑之靴', '雪地之靴', '射手护腕', '护盾胸针', '飞天扫帚', '水下呼吸帽', '爆裂头饰', '精灵斗篷', '防护斗篷', '蝠鲼斗篷', '无尽水瓶', '幻象牌组', '漂浮之球', '无踪粉', '干燥粉', '喷嚏粉', '元素宝石', '无尽烟壶', '魅惑镜片', '微视镜片', '鹰眼镜片', '食人魔力量护手', '光彩夺目宝石', '辟矢夺箭手套', '运动健将手套', '窃贼手套', '夜视镜', '易容帽', '智力头带', '通晓语言头盔', '心灵感应头盔', '不动权杖', '闪电标枪', '卡夫统灵药', '显像提灯', '水手护甲', '读心勋章', '秘银甲', '适应项链', '滑溜之油', '法力再生珍珠', '保健护符', '愈合护符', '迷情媚药', '颤栗乐笙', '唤鼠乐笙', '化兽为友药水', '火焰吐息药水', '成长药水', '剧毒药水', '抗性药水', '水下呼吸药水', '艾罗娜的箭袋', '跳跃戒指', '心灵护盾戒指', '善泳戒指', '温暖戒指', '水行戒指', '杂货法袍', '攀爬绳', '铁骑马鞍', '传讯石', '警戒之盾', '蛛行便鞋', '蝰蛇法杖', '巨蟒法杖', '幸运石', '复仇之剑', '唤鱼三叉戟', '搜魔魔杖', '魔法飞弹魔杖', '探秘魔杖', '蛛网魔杖', '警戒武器', '造风之扇', '飞翼之靴', '健康护符', '抗性护甲', '易伤护甲', '吸矢盾', '魔豆之袋', '力场珠', '矮人腰带', '狂战斧', '浮空之靴', '速度之靴', '命令元素水钵', '防御护腕', '命令元素火盆', '郎中披肩', '命令元素香炉', '开门钟琴', '移位斗篷', '蝙蝠斗篷', '力场魔方', '迪恩的折叠要塞', '淬毒匕首', '次元镣铐', '屠龙者', '健康灵药', '精灵链甲', '焰舌', '折叠船', '真视宝石', '巨人屠杀者', '幻惑镶钉皮甲', '传送头盔', '霍华德的便利袋', '音爆号角', '加速马蹄铁', '比拉罗的铁索', '瓦解之锤', '打击之锤', 